#### 1. Import pandas library

In [14]:
import pandas as pd

#### 2. Import users table:

In [15]:
users_table = pd.read_csv('users_table.csv')
users_table.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,NaN


#### 3. Rename Id column to userId

In [16]:
users_table = users_table.rename(columns={'Id': 'userId'})

#### 4. Import posts table:

In [17]:
posts_table = pd.read_csv('posts_table.csv')

#### 5. Rename Id column to postId and OwnerUserId to userId

In [18]:
posts_table = posts_table.rename(columns={'Id': 'postId' , 'OwnerUserId': 'userId'})

#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [19]:
users = users_table[['userId', 'Reputation', 'Views', 'UpVotes', 'DownVotes']]
posts = posts_table[['postId', 'Score', 'userId', 'ViewCount', 'CommentCount']]

#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [20]:
merged_table = pd.merge(users, posts)
merged_table.head()

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0
3,-1,1,0,5007,1920,8981,0,NaN,0
4,-1,1,0,5007,1920,8982,0,NaN,0


In [21]:
new_dataframeTest = posts.merge(users, left_on='userId', right_on='userId')
new_dataframeTest

,postId,Score,userId,ViewCount,CommentCount,Reputation,Views,UpVotes,DownVotes
0,1,23,8.0,1278.0,1,6764,1089,604,25
1,16,16,8.0,NaN,3,6764,1089,604,25
2,36,41,8.0,67396.0,7,6764,1089,604,25
3,65,14,8.0,NaN,3,6764,1089,604,25
4,78,33,8.0,NaN,4,6764,1089,604,25
...,...,...,...,...,...,...,...,...,...
38957,48313,-1,19735.0,NaN,1,1,5,0,0
38958,48317,8,19960.0,1894.0,1,153,9,18,0
38959,48318,0,19962.0,52.0,4,1,1,0,0
38960,48321,0,19966.0,NaN,0,1,1,0,0


#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [22]:
null_cols = merged_table.isnull().sum()
null_cols
null_cols[null_cols > 0]

ViewCount    23572
dtype: int64

In [23]:
null_cols

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64

In [24]:
null_cols = merged_table.isnull().sum()
null_cols[null_cols > 0]




ViewCount    23572
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [30]:
null_cols[['ViewCount']] = null_cols[['ViewCount']].fillna(0)
null_cols

# why this shit doesn't want to fill the the empty fields? it drives me crazy

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64

In [32]:
null_cols.fillna(0)

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64

In [27]:
merged_table.head()


,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0
3,-1,1,0,5007,1920,8981,0,NaN,0
4,-1,1,0,5007,1920,8982,0,NaN,0


In [31]:
null_cols = merged_table.isnull().sum()
null_cols[null_cols > 0]

ViewCount    23572
dtype: int64

In [28]:
merged_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38962 entries, 0 to 38961
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   userId        38962 non-null  int64  
 1   Reputation    38962 non-null  int64  
 2   Views         38962 non-null  int64  
 3   UpVotes       38962 non-null  int64  
 4   DownVotes     38962 non-null  int64  
 5   postId        38962 non-null  int64  
 6   Score         38962 non-null  int64  
 7   ViewCount     15390 non-null  float64
 8   CommentCount  38962 non-null  int64  
dtypes: float64(1), int64(8)
memory usage: 3.0 MB


#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [33]:
stats = merged_table.describe().transpose()
stats.head()

,count,mean,std,min,25%,50%,75%,max
userId,38962.0,6079.063087,5224.896435,-1.0,1317.0,4856.0,9651.0,55226.0
Reputation,38962.0,7281.091679,15164.527714,1.0,147.0,909.0,7931.0,87393.0
Views,38962.0,1400.648016,3423.886887,0.0,16.0,124.0,1050.0,20932.0
UpVotes,38962.0,914.799677,2296.527060,0.0,4.0,65.0,582.0,11442.0
DownVotes,38962.0,43.841050,161.797079,0.0,0.0,1.0,16.0,1920.0


In [35]:
stats['IQR'] = stats['75%'] - stats['25%']
stats

,count,mean,std,min,25%,50%,75%,max,IQR
userId,38962.0,6079.063087,5224.896435,-1.0,1317.00,4856.0,9651.0,55226.0,8334.00
Reputation,38962.0,7281.091679,15164.527714,1.0,147.00,909.0,7931.0,87393.0,7784.00
Views,38962.0,1400.648016,3423.886887,0.0,16.00,124.0,1050.0,20932.0,1034.00
UpVotes,38962.0,914.799677,2296.527060,0.0,4.00,65.0,582.0,11442.0,578.00
DownVotes,38962.0,43.841050,161.797079,0.0,0.00,1.0,16.0,1920.0,16.00
postId,38962.0,22960.799651,13696.932471,1.0,11325.25,22373.5,33688.5,48325.0,22363.25
Score,38962.0,4.083081,6.561843,-19.0,1.00,2.0,5.0,192.0,4.00
ViewCount,15390.0,1196.225991,3742.952815,14.0,173.00,385.0,967.0,175495.0,794.00
CommentCount,38962.0,2.014630,2.674018,0.0,0.00,1.0,3.0,45.0,3.00
